In [1]:
%matplotlib inline
# imports required packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pynetlogo
import numpy as np
import sys
sys.path.append('../')

from LandLab_experiments.ecohydr_mod import EcoHyd

In [ ]:
def get_yearly_temp(csv_path, num_years):
    df = pd.read_csv(csv_path)
    df.dt = pd.to_datetime(df.dt)
    avg_temp_per_year = []
    max_temp_per_year = []
    min_temp_per_year = []
    for year in range(0, num_years):
        earliest_yr = df.iloc[0, 0].year
        year = year + earliest_yr
        if len(df.loc[(df.dt.dt.year == year), :]) > 365:
            avg_temp_per_year.append(df.loc[(df.dt.dt.year == year), 'AverageTemperature'].iloc[:-1].tolist())
            max_temp_per_year.append(df.loc[(df.dt.dt.year == year), 'MaxTemperature'].iloc[:-1].tolist())
            min_temp_per_year.append(df.loc[(df.dt.dt.year == year), 'MinTemperature'].iloc[:-1].tolist())
        else:
            avg_temp_per_year.append(df.loc[(df.dt.dt.year == year), 'AverageTemperature'].tolist())
            max_temp_per_year.append(df.loc[(df.dt.dt.year == year), 'MaxTemperature'].tolist())
            min_temp_per_year.append(df.loc[(df.dt.dt.year == year), 'MinTemperature'].tolist())
    
    return avg_temp_per_year, max_temp_per_year, min_temp_per_year

In [11]:
def setUpNetLogoModel(leadFarmers, desperation, jealousy, grace):
    # think this is for the GUI idk?
    sns.set_style("white")
    sns.set_context("talk")

    # starts a NetLogo link, point the netlogo home path to where you have it installed on your machine
    netlogo = pynetlogo.NetLogoLink(
        gui=False,
        netlogo_home="/Volumes/NetLogo 6.3.0/NetLogo 6.3.0"
    )

    # loads a .nlogo model from provided path
    netlogo.load_model("/Users/mabelwylie/Documents/NetLogo-MOSS-Python/CouplingRepo/TreesAndThat/Coupling/modelv1.nlogo")

     # sets globals
    globals = "update-globals " + str(leadFarmers) + " " + str(desperation) + " " + str(jealousy) + " " + str(grace)
    netlogo.command(globals)
    
    # runs the model setup command
    netlogo.command("setup")

    return netlogo

In [6]:
def reportsToDataFrame(netlogo):
    # gets field attributes and puts it in a data frame
    fieldAttributes = netlogo.report("get-info")
    sorted_list = sorted(fieldAttributes, key=lambda x: (-x[1],x[0]))
    fieldData = pd.DataFrame(columns=["who", "xcor","ycor","owner-id","implements-WSA", "yield"], data=sorted_list)
    return fieldData

In [7]:
def convertWSAToNPArray(data):
    # sets bool into correct format to pass to hydrology model
    emptynp = np.empty((51,51))
    for i in range(0,51):
        for j in range(0,51):
            ycorFromIndex = i * -1 + 25
            xcorFromIndex = j - 25
            emptynp[i][j]= data.loc[(data['ycor'] == ycorFromIndex) & (data['xcor'] == xcorFromIndex)]["implements-WSA"].iloc[0]
    return emptynp

In [8]:
def convertHydrologyToDF(hydrologyArray, data):
    # method converts the hydrology model output into a pandas dataframer
    flattenedArray = hydrologyArray.flatten()
    # 2 is the placeholder to multiply the biomass to get yield
    data["yield"] = flattenedArray * 2
    return data

In [12]:
def fullModelRun(leadFarmers, desperation, jealousy, grace, fileSuffix, input_csv_path, from_year, to_year):
    # sets up model
    netlogo = setUpNetLogoModel(leadFarmers, desperation, jealousy, grace)

    # gets base field attributes and puts it in a data frame
    fieldData = reportsToDataFrame(netlogo)

    # make dataframe to record all changes
    bestData = fieldData.copy()
    
    #get input temperature data
    avg, maxi, mini = get_yearly_temp(input_csv_path, from_year, to_year)

    for year in range(0,10):
        # run for 10 years

        avg_yr, maxi_yr, mini_yr = avg[year], maxi[year], mini[year]

        # runs one step of social model
        netlogo.command("farming-year")
        print("farming command")

        # converts field data to a df
        fieldData = reportsToDataFrame(netlogo)
        print("convertedtoDF")

        # converts the usingWSA bool for each field into an NP array
        hydrologyInput = convertWSAToNPArray(fieldData)
        print("converted to NP array")

        # use dummy model
        hydrologyOutput = hydrologyModelYear(hydrologyInput, avg_yr, maxi_yr, mini_yr)
        
        # trying to implement the hydrology model
        # input to this block will be the WSA np.array, with 0 or 1 corresponding to if the field is using WSA
        # the output of this block should be a np.array with biomass
        
        #Ecohyd_model = EcoHyd(WSA_array=hydrologyInput)
        #EcoHyd.stepper(Ecohyd_model)

        # converts the updated yields to the dataframe
        convertHydrologyToDF(hydrologyOutput,fieldData)
        print("Converted output to df")

        # writes this new yield information to the netlogo implementation
        netlogo.write_NetLogo_attriblist(fieldData, "field")
        print("wrote to netlogo")

        

## dummy hydrology model to test running
def hydrologyModelYear(input):
    print("One year yield")
    return np.zeros((51,51))

fullModelRun(20, 0.5, 0.5, 3, 1)

farming command
convertedtoDF
converted to NP array
One year yield
Converted output to df
wrote to netlogo
farming command
convertedtoDF
converted to NP array
One year yield
Converted output to df
wrote to netlogo
farming command
convertedtoDF
converted to NP array
One year yield
Converted output to df
wrote to netlogo
farming command
convertedtoDF
converted to NP array
One year yield
Converted output to df
wrote to netlogo
farming command
convertedtoDF
converted to NP array
One year yield
Converted output to df
wrote to netlogo
farming command
convertedtoDF
converted to NP array
One year yield
Converted output to df
wrote to netlogo
farming command
convertedtoDF
converted to NP array
One year yield
Converted output to df
wrote to netlogo
farming command
convertedtoDF
converted to NP array
One year yield
Converted output to df
wrote to netlogo
farming command
convertedtoDF
converted to NP array
One year yield
Converted output to df
wrote to netlogo
farming command
convertedtoDF
convert